In [118]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
#driver2.quit()

In [2]:
driver2 = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/abigailasper/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_62558/3351975099.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome(ChromeDriverManager().install())


## log in to GoodReads

In [3]:
driver2.get("https://www.goodreads.com/user/sign_in")

In [4]:
#select "sign in with email" option
#https://www.geeksforgeeks.org/click-button-by-text-using-python-and-selenium/
button = driver2.find_element_by_link_text("Sign in with email")
button.click()

/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_62558/2392284598.py:3: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  button = driver2.find_element_by_link_text("Sign in with email")


In [5]:
username = driver2.find_element(By.NAME, "email")
username.send_keys("abigailasper@gmail.com")

### import json & keys file to safely import GoodReads password

In [6]:
import json

In [7]:
with open('../keys/gr_pw.json') as fi:
    credentials = json.load(fi)

In [8]:
gr_pw = credentials['gr_pw']

In [9]:
password = driver2.find_element(By.NAME, "password")
password.send_keys(gr_pw)

In [10]:
#submit login credentials
signin = driver2.find_element(By.ID, "signInSubmit")
signin.click()

## navigate to desired shelf (Booktok Books) & begin scrape

In [11]:
driver2.get("https://www.goodreads.com/shelf/show/booktok")

In [12]:
#https://www.geeksforgeeks.org/how-to-scrape-multiple-pages-using-selenium-in-python/
book_list = []
urls = []
meta_data = []

for page in range(1, 26, 1):
    
    page_url = "https://www.goodreads.com/shelf/show/booktok?page=" + str(page)
    driver2.get(page_url)
    soup = BS(driver2.page_source, 'html.parser')
    title = driver2.find_elements(By.CLASS_NAME, "bookTitle")
    author = driver2.find_elements(By.CLASS_NAME, "authorName")
    #https://stackoverflow.com/questions/29078072/how-to-get-link-value-inside-href
    #https://selenium-python.readthedocs.io/locating-elements.html
    hrefs = driver2.find_elements(By.XPATH, "//a[@class='bookTitle']")
    cards = soup.find_all("div", {"class": "elementList"})
  
    for card in cards:
        meta_data.append(str(card.find(string=re.compile("avg rating"))))
  
    for my_href in hrefs:
        urls.append(my_href.get_attribute("href"))
    
    for i in range(len(title)):
        book_list.append([title[i].text, author[i].text])
  
print(book_list)
        

[['The Seven Husbands of Evelyn Hugo (Hardcover)', 'Taylor Jenkins Reid'], ['It Ends with Us (Kindle Edition)', 'Colleen Hoover'], ['The Song of Achilles (Paperback)', 'Madeline Miller'], ['The Invisible Life of Addie LaRue (Hardcover)', 'V.E. Schwab'], ['The Cruel Prince (The Folk of the Air, #1)', 'Holly Black'], ['From Blood and Ash (Blood and Ash, #1)', 'Jennifer L. Armentrout'], ['Six of Crows (Six of Crows, #1)', 'Leigh Bardugo'], ['They Both Die at the End (Hardcover)', 'Adam Silvera'], ['A Court of Thorns and Roses (A Court of Thorns and Roses, #1)', 'Sarah J. Maas'], ['Red, White & Royal Blue (Paperback)', 'Casey McQuiston'], ['The Love Hypothesis (Paperback)', 'Ali Hazelwood'], ['Ugly Love (Kindle Edition)', 'Colleen Hoover'], ['We Were Liars (Kindle Edition)', 'E. Lockhart'], ['People We Meet on Vacation (Paperback)', 'Emily Henry'], ['The Spanish Love Deception (ebook)', 'Elena Armas'], ['The Hating Game (ebook)', 'Sally Thorne'], ['Beach Read (Paperback)', 'Emily Henry'], 

In [148]:
#make list a dataframe
books_df = pd.DataFrame(book_list)

books_df.head()

,0,1
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid
1,It Ends with Us (Kindle Edition),Colleen Hoover
2,The Song of Achilles (Paperback),Madeline Miller
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black


In [149]:
#rename columns
books_df.columns = ['title', 'author']

In [150]:
#remove empty values from list
#https://www.geeksforgeeks.org/python-remove-empty-strings-from-list-of-strings/
while("None" in meta_data) :
    meta_data.remove("None")

In [151]:
#add urls & meta_data to dataframe
books_df['urls'] = urls
books_df['meta_data'] = meta_data

books_df.head()

,title,author,urls,meta_data
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,https://www.goodreads.com/book/show/32620332-the-seven-husbands-of-evelyn-hugo,"\n avg rating 4.48 —\n 1,155,356 ratings —\n published 2017\n"
1,It Ends with Us (Kindle Edition),Colleen Hoover,https://www.goodreads.com/book/show/27362503-it-ends-with-us,"\n avg rating 4.42 —\n 1,149,821 ratings —\n published 2016\n"
2,The Song of Achilles (Paperback),Madeline Miller,https://www.goodreads.com/book/show/13623848-the-song-of-achilles,"\n avg rating 4.40 —\n 815,625 ratings —\n published 2011\n"
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab,https://www.goodreads.com/book/show/50623864-the-invisible-life-of-addie-larue,"\n avg rating 4.23 —\n 603,813 ratings —\n published 2020\n"
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,https://www.goodreads.com/book/show/26032825-the-cruel-prince,"\n avg rating 4.10 —\n 629,204 ratings —\n published 2018\n"


In [152]:
#checking to make sure final book id matches final book - it does!
books_df.tail()

,title,author,urls,meta_data
1245,"A Magic Steeped in Poison (The Book of Tea, #1)",Judy I. Lin,https://www.goodreads.com/book/show/56978089-a-magic-steeped-in-poison,"\n avg rating 3.96 —\n 3,919 ratings —\n published 2022\n"
1246,"The Brazen (Calamity Montana, #3)",Willa Nash,https://www.goodreads.com/book/show/56682105-the-brazen,"\n avg rating 4.21 —\n 4,597 ratings —\n published 2021\n"
1247,"River of Shadows (Underworld Gods, #1)",Karina Halle,https://www.goodreads.com/book/show/59531495-river-of-shadows,"\n avg rating 3.94 —\n 3,490 ratings —\n published 2022\n"
1248,Sinners Anonymous (Sinners Anonymous #1),Somme Sketcher,https://www.goodreads.com/book/show/59818980-sinners-anonymous,\n avg rating 4.21 —\n 930 ratings —\n published \n
1249,Cruel Princes (Princes of Devil's Creek Book 1),Jillian Frost,https://www.goodreads.com/book/show/60818779-cruel-princes,"\n avg rating 3.64 —\n 2,124 ratings —\n published \n"


In [153]:
#trim /n line breaks from beginning and end of meta_data
#https://stackoverflow.com/questions/275018/how-do-i-remove-a-trailing-newline#:~:text=You%20may%20use%20line%20%3D%20line,the%20string%2C%20not%20just%20one.
books_df['meta_data'] = books_df['meta_data'].str.strip('\n')
books_df.head()

,title,author,urls,meta_data
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,https://www.goodreads.com/book/show/32620332-the-seven-husbands-of-evelyn-hugo,"avg rating 4.48 —\n 1,155,356 ratings —\n published 2017\n"
1,It Ends with Us (Kindle Edition),Colleen Hoover,https://www.goodreads.com/book/show/27362503-it-ends-with-us,"avg rating 4.42 —\n 1,149,821 ratings —\n published 2016\n"
2,The Song of Achilles (Paperback),Madeline Miller,https://www.goodreads.com/book/show/13623848-the-song-of-achilles,"avg rating 4.40 —\n 815,625 ratings —\n published 2011\n"
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab,https://www.goodreads.com/book/show/50623864-the-invisible-life-of-addie-larue,"avg rating 4.23 —\n 603,813 ratings —\n published 2020\n"
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,https://www.goodreads.com/book/show/26032825-the-cruel-prince,"avg rating 4.10 —\n 629,204 ratings —\n published 2018\n"


In [154]:
#expand meta_data into individual columns
#https://datascienceparichay.com/article/pandas-split-column-by-delimiter/
#https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html#pandas.Series.str.split
meta_data_df = books_df['meta_data'].str.split(pat="\n", expand=True)
meta_data_df.head()

,0,1,2,3
0,avg rating 4.48 —,"1,155,356 ratings —",published 2017,
1,avg rating 4.42 —,"1,149,821 ratings —",published 2016,
2,avg rating 4.40 —,"815,625 ratings —",published 2011,
3,avg rating 4.23 —,"603,813 ratings —",published 2020,
4,avg rating 4.10 —,"629,204 ratings —",published 2018,


In [155]:
#rename metadata columns
meta_data_df.columns = ['avg_rating', 'num_ratings', 'date_published', 'x']
meta_data_df.head()

,avg_rating,num_ratings,date_published,x
0,avg rating 4.48 —,"1,155,356 ratings —",published 2017,
1,avg rating 4.42 —,"1,149,821 ratings —",published 2016,
2,avg rating 4.40 —,"815,625 ratings —",published 2011,
3,avg rating 4.23 —,"603,813 ratings —",published 2020,
4,avg rating 4.10 —,"629,204 ratings —",published 2018,


In [156]:
#drop empty column
meta_data_df = meta_data_df.drop(['x'], axis=1)
meta_data_df.head()

,avg_rating,num_ratings,date_published
0,avg rating 4.48 —,"1,155,356 ratings —",published 2017
1,avg rating 4.42 —,"1,149,821 ratings —",published 2016
2,avg rating 4.40 —,"815,625 ratings —",published 2011
3,avg rating 4.23 —,"603,813 ratings —",published 2020
4,avg rating 4.10 —,"629,204 ratings —",published 2018


In [158]:
#clean up date_published column
meta_data_df['date_published'] = meta_data_df['date_published'].str[-4:]
meta_data_df.head()

,avg_rating,num_ratings,date_published
0,avg rating 4.48 —,"1,155,356 ratings —",2017
1,avg rating 4.42 —,"1,149,821 ratings —",2016
2,avg rating 4.40 —,"815,625 ratings —",2011
3,avg rating 4.23 —,"603,813 ratings —",2020
4,avg rating 4.10 —,"629,204 ratings —",2018


In [159]:
#clean up avg_rating column
meta_data_df['avg_rating'] = meta_data_df['avg_rating'].astype(str)
meta_data_df['avg_rating'] = meta_data_df['avg_rating'].str.lstrip('avg rating ')
meta_data_df['avg_rating'] = meta_data_df['avg_rating'].str.rstrip(' —')
meta_data_df.head()

,avg_rating,num_ratings,date_published
0,4.48,"1,155,356 ratings —",2017
1,4.42,"1,149,821 ratings —",2016
2,4.40,"815,625 ratings —",2011
3,4.23,"603,813 ratings —",2020
4,4.10,"629,204 ratings —",2018


In [160]:
#clean up num_ratings column
meta_data_df['num_ratings'] = meta_data_df['num_ratings'].astype(str)
meta_data_df['num_ratings'] = meta_data_df['num_ratings'].str.rstrip(' ratings —')
meta_data_df.head()

,avg_rating,num_ratings,date_published
0,4.48,"1,155,356",2017
1,4.42,"1,149,821",2016
2,4.40,"815,625",2011
3,4.23,"603,813",2020
4,4.10,"629,204",2018


In [161]:
#join books_df and meta_data_df
book_data = books_df.join(meta_data_df)
book_data.head()

,title,author,urls,meta_data,avg_rating,num_ratings,date_published
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,https://www.goodreads.com/book/show/32620332-the-seven-husbands-of-evelyn-hugo,"avg rating 4.48 —\n 1,155,356 ratings —\n published 2017\n",4.48,"1,155,356",2017
1,It Ends with Us (Kindle Edition),Colleen Hoover,https://www.goodreads.com/book/show/27362503-it-ends-with-us,"avg rating 4.42 —\n 1,149,821 ratings —\n published 2016\n",4.42,"1,149,821",2016
2,The Song of Achilles (Paperback),Madeline Miller,https://www.goodreads.com/book/show/13623848-the-song-of-achilles,"avg rating 4.40 —\n 815,625 ratings —\n published 2011\n",4.40,"815,625",2011
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab,https://www.goodreads.com/book/show/50623864-the-invisible-life-of-addie-larue,"avg rating 4.23 —\n 603,813 ratings —\n published 2020\n",4.23,"603,813",2020
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,https://www.goodreads.com/book/show/26032825-the-cruel-prince,"avg rating 4.10 —\n 629,204 ratings —\n published 2018\n",4.10,"629,204",2018


In [162]:
#drop meta_data column
book_data = book_data.drop(['meta_data'], axis=1)

book_data.head()

,title,author,urls,avg_rating,num_ratings,date_published
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,https://www.goodreads.com/book/show/32620332-the-seven-husbands-of-evelyn-hugo,4.48,"1,155,356",2017
1,It Ends with Us (Kindle Edition),Colleen Hoover,https://www.goodreads.com/book/show/27362503-it-ends-with-us,4.42,"1,149,821",2016
2,The Song of Achilles (Paperback),Madeline Miller,https://www.goodreads.com/book/show/13623848-the-song-of-achilles,4.40,"815,625",2011
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab,https://www.goodreads.com/book/show/50623864-the-invisible-life-of-addie-larue,4.23,"603,813",2020
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,https://www.goodreads.com/book/show/26032825-the-cruel-prince,4.10,"629,204",2018


In [163]:
#add column that ranks books by row, as books are currently in order of # times shelved as "booktok" (most often to. least often)

book_data['booktok_popularity'] = np.arange(len(book_data))
book_data.head()

,title,author,urls,avg_rating,num_ratings,date_published,booktok_popularity
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,https://www.goodreads.com/book/show/32620332-the-seven-husbands-of-evelyn-hugo,4.48,"1,155,356",2017,0
1,It Ends with Us (Kindle Edition),Colleen Hoover,https://www.goodreads.com/book/show/27362503-it-ends-with-us,4.42,"1,149,821",2016,1
2,The Song of Achilles (Paperback),Madeline Miller,https://www.goodreads.com/book/show/13623848-the-song-of-achilles,4.40,"815,625",2011,2
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab,https://www.goodreads.com/book/show/50623864-the-invisible-life-of-addie-larue,4.23,"603,813",2020,3
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,https://www.goodreads.com/book/show/26032825-the-cruel-prince,4.10,"629,204",2018,4


In [164]:
#save dataframe as csv
book_data.to_csv('../data/booktok_books.csv', index=False)